In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import mglearn

In [2]:
df = pd.read_csv("./data/breast-cancer.data", header=None)
df = df.rename(columns={
    0: "Label", 
    1: "Age", 
    2: "Menopause",
    3: "Tumor Size",
    4: "Inv Nodes",
    5: "Node Caps",
    6: "Malignance Degree",
    7: "Breast",
    8: "Breast Quadrant",
    9: "Irradiated"
})
df.head(15)

,Label,Age,Menopause,Tumor Size,Inv Nodes,Node Caps,Malignance Degree,Breast,Breast Quadrant,Irradiated
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no
5,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,left,left_low,no
6,no-recurrence-events,50-59,premeno,25-29,0-2,no,2,left,left_low,no
7,no-recurrence-events,60-69,ge40,20-24,0-2,no,1,left,left_low,no
8,no-recurrence-events,40-49,premeno,50-54,0-2,no,2,left,left_low,no
9,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,left_up,no


In [3]:
df = df[df["Breast Quadrant"] != "?"]
df = df[df["Node Caps"] != "?"]


def get_label(label):
    if label == "no-recurrence-events":
        return 1
    return 0


def get_age(age_range):
    return int(age_range.split("-")[0])


def get_menopause(menopause):
    if menopause == "premeno":
        return 1
    elif menopause == "ge40":
        return 2
    else:
        return 3


def get_tumor_size(size_range):
    return int(size_range.split("-")[1])


def get_inv_nodes(inv_range):
    return int(inv_range.split("-")[1])
    
    
def get_node_caps(caps):
    if caps == "yes":
        return 1
    else:
        return 0
    

def get_breast(breast):
    if breast == "left":
        return 1
    else:
        return 0
    

def get_breast_quad(quad):
    if quad == "left_low":
        return 1
    elif quad == "left_up":
        return 2
    elif quad == "right_low":
        return 3
    elif quad == "right_up":
        return 4
    else:
        return 5
    
    
def get_irradiated(irr):
    if irr == "yes":
        return 1
    else:
        return 0

    
df["Label"] = df["Label"].apply(lambda x: get_label(x))
df["Age"] = df["Age"].apply(lambda x: get_age(x))
df["Menopause"] = df["Menopause"].apply(lambda x: get_menopause(x))
df["Tumor Size"] = df["Tumor Size"].apply(lambda x: get_tumor_size(x))
df["Inv Nodes"] = df["Inv Nodes"].apply(lambda x: get_inv_nodes(x))
df["Node Caps"] = df["Node Caps"].apply(lambda x: get_node_caps(x))
df["Breast"] = df["Breast"].apply(lambda x: get_breast(x))
df["Breast Quadrant"] = df["Breast Quadrant"].apply(lambda x: get_breast(x))
df["Irradiated"] = df["Irradiated"].apply(lambda x: get_irradiated(x))
df.head(15)

,Label,Age,Menopause,Tumor Size,Inv Nodes,Node Caps,Malignance Degree,Breast,Breast Quadrant,Irradiated
0,1,30,1,34,2,0,3,1,0,0
1,1,40,1,24,2,0,2,0,0,0
2,1,40,1,24,2,0,2,1,0,0
3,1,60,2,19,2,0,2,0,0,0
4,1,40,1,4,2,0,2,0,0,0
5,1,60,2,19,2,0,2,1,0,0
6,1,50,1,29,2,0,2,1,0,0
7,1,60,2,24,2,0,1,1,0,0
8,1,40,1,54,2,0,2,1,0,0
9,1,40,1,24,2,0,2,0,0,0


In [22]:
df_X = df.drop("Label", axis=1)
df_y = df["Label"]
X, y = df_X.to_numpy(), df_y.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=df_y, random_state=42)
X_train.shape

(207, 9)

In [23]:
forest = RandomForestClassifier(n_estimators=5, random_state=42)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [30]:
y_pred = forest.predict(X_test)
total = X_test.shape[0]
accuracy = (100 * ((y_test == y_pred).sum() / total))
print(f"Accuracy = {round(accuracy, 2)}%")

Accuracy = 74.29%


In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(20, 10))
for i, (ax, tree) in enumerate(zip(axes.ravel(), forest.estimators_)):
    ax.set_title(f"Tree {i}")
    mglearn.plots.plot_tree_partition(X_train, y_train, tree, ax=ax)

mglearn.plots.plot_2d_separator(forest, X_train, fill=True, ax=axes[-1, -1], alpha=.4)
axes[-1, -1].set_title("Random Forest")
mglearn.discrete_scatter(X_train[:, 0], X_train[:, -1], y_train)